In [1]:
!pip install librosa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!git clone https://github.com/novoic/surfboard

fatal: destination path 'surfboard' already exists and is not an empty directory.


In [3]:
%cd surfboard

/content/surfboard


In [4]:
!pip install .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/surfboard
  Preparing metadata (setup.py) ... done
  Using cached numba-0.48.0-cp39-cp39-linux_x86_64.whl
  Using cached pyloudnorm-0.1.0-py3-none-any.whl (9.3 kB)
  Using cached pytest-7.2.2-py3-none-any.whl (317 kB)
  Using cached llvmlite-0.31.0.tar.gz (110 kB)
  Preparing metadata (setup.py) ... done
  Using cached exceptiongroup-1.1.1-py3-none-any.whl (14 kB)
  Using cached iniconfig-2.0.0-py3-none-any.whl (5.9 kB)
  Using cached pluggy-1.0.0-py2.py3-none-any.whl (13 kB)
  Using cached resampy-0.4.1-py3-none-any.whl (3.1 MB)
  Using cached resampy-0.4.0-py3-none-any.whl (3.1 MB)
  Using cached resampy-0.3.1-py3-none-any.whl (3.1 MB)
  Created wheel for surfboard: filename=surfboard-0.2.0-py3-none-any.whl size=43076 sha256=e1ca74d0d0b49f0f468fc33bd2af6f22d0dc4d4aec64918ed98336539136d17b
  Stored in directory: /tmp/pip-ephem-wheel-cache-1itk384l/wheels/88/dc/ca/6d2f

In [5]:
!pip install peakutils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
!pip install pyloudnorm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
!pip install pysptk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
import surfboard

In [9]:
import pandas as pd
import numpy as np


from google.colab import drive



from surfboard import feature_extraction

In [10]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
!pip install flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz

--2023-03-16 11:16:39--  https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 18.205.222.128, 54.161.241.46, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|18.205.222.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8162036 (7.8M) [application/octet-stream]
Saving to: ‘ngrok-v3-stable-linux-amd64.tgz.2’

ngrok-v3-stable-lin 100%[===================>]   7.78M  35.1MB/s    in 0.2s    

2023-03-16 11:16:39 (35.1 MB/s) - ‘ngrok-v3-stable-linux-amd64.tgz.2’ saved [8162036/8162036]



In [13]:
!tar -xzvf ngrok-v3-stable-linux-amd64.tgz

ngrok


In [14]:
!cp ngrok /usr/local/bin

In [15]:
!ngrok config add-authtoken 2N5VRFnDZCi53Qm0vSCYjcABA9u_5q1t1YrgLiech5q6edt23

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [16]:
from flask import Flask, jsonify, request
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)

import pickle

loaded_model = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/Project/model.pkl', 'rb'))


@app.route('/predict_pneumonia')
def predict_pneumonia():
    config = {'log_melspec': {'hop_length_seconds': 0.02, 'n_fft_seconds': 0.08,'n_mels': 64},
    'loudness_slidingwindow': {'frame_length_seconds': 1.0,'hop_length_seconds': 0.25}}
    path = request.args.get('path')
    X_temp = (
    feature_extraction.extract_features_from_paths(
        [path],
        components_list=[{key: config[key]} for key in config],
        statistics_list=["mean", "std"],
    )
    .replace(-np.inf, np.nan)
    .fillna(method="bfill"))
    y_temp = loaded_model.predict(X_temp)
    print(y_temp)
    return str(y_temp)

if __name__ == '__main__':
  app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://9780-35-229-21-194.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


Extracting features from paths...: 100%|██████████| 1/1 [00:02<00:00,  2.05s/it]
INFO:werkzeug:127.0.0.1 - - [16/Mar/2023 11:18:51] "GET /predict_pneumonia?path=/content/drive/MyDrive/Colab%20Notebooks/Project/pneumonia_files/audio_files/20190313-124456-b35bc39d3742c601-chest02.wav HTTP/1.1" 200 -


[0]


...

Extracting features from paths...: 100%|██████████| 1/1 [00:00<00:00,  1.32it/s]
INFO:werkzeug:127.0.0.1 - - [16/Mar/2023 11:22:16] "GET /predict_pneumonia?path=/content/drive/MyDrive/Colab%20Notebooks/Project/pneumonia_files/audio_files/20190313-124456-b35bc39d3742c601-chest02.wav HTTP/1.1" 200 -


[0]


Extracting features from paths...:   0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.9/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
Extracting features from paths...:   0%|          | 0/1 [00:00<?, ?it/s]
ERROR:__main__:Exception on /predict_pneumonia [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/librosa/core/audio.py", line 149, in load
    with sf.SoundFile(path) as sf_desc:
  File "/usr/local/lib/python3.9/dist-packages/soundfile.py", line 658, in __init__
    self._file = self._open(file, mode_int, closefd)
  File "/usr/local/lib/python3.9/dist-packages/soundfile.py", line 1216, in _open
    raise LibsndfileError(err, prefix="Error opening {0!r}: ".format(self.name))
soundfile.LibsndfileError: Error opening '/content/drive/MyDrive/Colab Notebooks/Project/pneumonia_files/audio_files/20190313-123957-4e78bbb32dd7e8c

[0]


Extracting features from paths...:   0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.9/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
Extracting features from paths...:   0%|          | 0/1 [00:00<?, ?it/s]
ERROR:__main__:Exception on /predict_pneumonia [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/librosa/core/audio.py", line 149, in load
    with sf.SoundFile(path) as sf_desc:
  File "/usr/local/lib/python3.9/dist-packages/soundfile.py", line 658, in __init__
    self._file = self._open(file, mode_int, closefd)
  File "/usr/local/lib/python3.9/dist-packages/soundfile.py", line 1216, in _open
    raise LibsndfileError(err, prefix="Error opening {0!r}: ".format(self.name))
soundfile.LibsndfileError: Error opening '/content/drive/MyDrive/Colab Notebooks/Project/pneumonia_files/audio_files/20190615-141414-a8b55ef899f4b51

In [17]:
!pip install flask --quiet
!pip install flask-ngrok --quiet
print("Completed!")


Completed!


In [18]:


# install ngrok linux version using the following command or you can get the
# latest version from its official website- https://dashboard.ngrok.com/get-started/setup

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz



--2023-03-16 11:46:46--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 54.237.133.81, 18.205.222.128, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13856790 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.tgz.2’

ngrok-stable-linux- 100%[===================>]  13.21M  55.2MB/s    in 0.2s    

2023-03-16 11:46:46 (55.2 MB/s) - ‘ngrok-stable-linux-amd64.tgz.2’ saved [13856790/13856790]



In [19]:

# extract the downloaded file using the following command 
!tar -xvf /content/ngrok-stable-linux-amd64.tgz


# paste your AuthToken here and execute this command
!./ngrok authtoken 2N5VRFnDZCi53Qm0vSCYjcABA9u_5q1t1YrgLiech5q6edt23


ngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [20]:

# import Flask from flask module
from flask import Flask, render_template, request

# import run_with_ngrok from flask_ngrok to run the app using ngrok
from flask_ngrok import run_with_ngrok
  
app = Flask(__name__) #app name
run_with_ngrok(app)
  
# @app.route("/")
# def hello():
#     return "Hello Friends! from Pykit.org. Thank you! for reading this article."

@app.route("/")
def home():
    return "hello"



if __name__ == "__main__":
    #app.config["UPLOAD_FOLDER"] = "/tmp"
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://b3e9-35-229-21-194.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
